In [ ]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.coils as jsc

from tests.coils.test_coils import _check_single_vectorized_internal
import pyvista as pv
from dataclasses import dataclass

from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[2]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

In [ ]:
class bb:
    param : str
    def __init__(self, param):
        self.param = param

def _get_all_discrete_coils(request):    
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)    
    coilset_jaxsbgeom = jsb.coils.CoilSet([jsb.coils.DiscreteCoil.from_positions(coilset_sbgeom[i].Get_Vertices()) for i in range(coilset_sbgeom.Number_of_Coils())])
    
    return coilset_jaxsbgeom, coilset_sbgeom


jsbc, sbgc = _get_all_discrete_coils(bb(coil_files[1]))

In [ ]:
from jax_sbgeom.coils.base_coil import CentroidFrame, FiniteSizeCoil, RotationMinimizedFrame


finite_size_method = RotationMinimizedFrame.from_coil(jsbc[0], 40)

fsc = FiniteSizeCoil(coil=jsbc[0], finite_size_method=finite_size_method)

In [ ]:
mesh_sb = sbgc[0].Mesh_Triangles_RMF(0.2,0.2, 40)
mesh_cj = jsb.coils.coil_meshing.mesh_coil_surface(fsc, 40, 0.2, 0.2)

onp.testing.assert_allclose(mesh_sb.vertices, mesh_cj[0], rtol=1e-12, atol=1e-12)

In [ ]:
plotter =  pv.Plotter()


pvmesh_sb = sbgc[0].Mesh_Triangles_RMF(0.6,0.2, 100).to_pyvista()
pvmesh = _mesh_to_pyvista_mesh(*jsb.coils.coil_meshing.mesh_coil_surface(fsc, 100, 0.2, 0.6))
plotter.add_mesh(pvmesh_sb, color="blue", opacity=1.0, show_edges=True)
plotter.add_mesh(pvmesh, color="red", opacity=1.0, show_edges=True)
plotter.show()